# IMPORTING USEFUL PYTHON LIBRARIES

In [154]:
#Importing useful libraries in Python
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas, xgboost, numpy, string
import pandas as pd
import numpy as np
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [153]:
! pip install tensorflow

    100% |████████████████████████████████| 62.0MB 410kB/s ta 0:00:011    32% |██████████▌                     | 20.3MB 6.7MB/s eta 0:00:07    38% |████████████▍                   | 24.1MB 8.6MB/s eta 0:00:05
    100% |████████████████████████████████| 1.2MB 4.6MB/s ta 0:00:011
    100% |████████████████████████████████| 3.1MB 3.9MB/s eta 0:00:01
    100% |████████████████████████████████| 102kB 10.9MB/s a 0:00:01
    100% |████████████████████████████████| 2.0MB 5.8MB/s eta 0:00:01   21% |███████                         | 440kB 5.4MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 8.9MB/s ta 0:00:011
  Running setup.py bdist_wheel for gast ... done
  Stored in directory: /Users/paramawasthi/Library/Caches/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /Users/paramawasthi/Library/Caches/pip/wheels/18/ea/5e/e36e1b8739e78cd2eba0a08fdc602c2b16a4b263912af8cb64
  Running setup.p

In [7]:
import multiprocessing
n_jobs_cnt = multiprocessing.cpu_count()-1
n_jobs_cnt

3

# IMPORTING THE DATA AND DATA PREPROCESSING

In [8]:
#importing train review text and label files to dataframe
import pandas as pd
df_test_text = pd.read_table('imdb_test_text.txt', delim_whitespace=False, names=('A'))
df_test_labels = pd.read_table('imdb_test_labels.txt', delim_whitespace=False, names=('B'))

In [9]:
#merging them on index
df_test=df_test_text.join(df_test_labels, how='outer')
df_test.shape

(25000, 2)

In [17]:
#importing test review text and label files to dataframe
df_train_text = pd.read_table('imdb_train_text.txt', delim_whitespace=False, names=('A'))
df_train_labels = pd.read_table('imdb_train_labels.txt', delim_whitespace=False, names=('B'))

In [18]:
#merging them on index
df_train=df_train_text.join(df_train_labels, how='outer')

In [118]:
#changing column name to review_text and review_label
df_train=df_train.rename(columns={'A':'review_text',
                          'B':'review_label'})
df_train.index.name='S.No.'
df_train.head()

,review_text,review_label
S.No.,,
0,I loved this movie since I was 7 and I saw it ...,10
1,"First things first, Edison Chen did a fantasti...",8
2,"Once again, I was browsing through the discoun...",7
3,"This is a gem, a real piece of Americana for a...",8
4,While I had wanted to se this film since the f...,8


In [119]:
#changing column name to review_text and review_label
df_test=df_test.rename(columns={'A':'review_text',
                          'B':'review_label'})
df_test.index.name='S.No.'
df_test.head(10)

,review_text,review_label
S.No.,,
0,not really sure what to make of this movie. ve...,7
1,"If you enjoyed films like Pulp Fiction, Reserv...",10
2,"Okay, here's the deal. There's this American p...",10
3,The BBC surpassed themselves with the boundari...,10
4,"Victor Mature, as a barely civilized and mostl...",8
5,I remember this film as the other person that ...,10
6,I first saw Heimat 2 on BBC2 in the 90's when ...,10
7,A rich experience is to be gained from watchin...,9
8,The second (not animated) movie about the only...,9


In [120]:
df_train.shape

(25000, 2)

In [121]:
#concatenating two datasets
df_all = pd.concat(objs=[df_train,
                         df_test],
                   axis=0)
df_all.reset_index(inplace=True)
df_all.shape

(50000, 3)

In [122]:
# Droping serial_no column
df_all.drop(labels=['S.No.'],
            inplace=True,
            axis=1)
df_all.head()


,review_text,review_label
0,I loved this movie since I was 7 and I saw it ...,10
1,"First things first, Edison Chen did a fantasti...",8
2,"Once again, I was browsing through the discoun...",7
3,"This is a gem, a real piece of Americana for a...",8
4,While I had wanted to se this film since the f...,8


# PREPARING THE DATASETS FOR MODEL FITTING

In [27]:
#Splitting the dataset into test and train for implementing ML models 
#mention a random split
X_train, X_test, y_train, y_test = train_test_split(df_all['review_text'],
                                                    df_all['review_label'],
                                                    test_size=0.2,
                                                    random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((40000,), (10000,), (40000,), (10000,))

In [123]:
#checking unique labels present in test and train data
y_train.unique(), y_test.unique()

(array([ 3,  8,  2,  1, 10,  4,  7,  9]),
 array([ 9, 10,  8,  3,  1,  4,  7,  2]))

In [29]:
# label encode the target variable

encoder = LabelEncoder()
y_train_en = encoder.fit_transform(y_train)
y_test_en = encoder.transform(y_test)

y_train_en.shape, y_test_en.shape

((40000,), (10000,))

## Starting to fit different ML models one by one

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, y_test_en)

###  We will now try to run fit our data through various ML models. The models will be implemented after vectorization, mainky tf-idf or count-vectorizer or both. 
We have used following models with different vectorization parameters to achieve highest accuracy:

1. Naive Bayes
2. Neural Network
3. Random Forest Classifier
4. XG Boost

# 1. Naive Bayes

### IMPLEMENTING TF-IDF VECTORIZER AND COUNT VECTORIZER 

### This was done in multiple iterations with different parameter values to achieve maximum accuracy on the results

### TF-DIF ON WORD LEVEL, NGRAM LEVEL, CHARS LEVEL

In [68]:
# word level tf-idf
%%time

tfidf_vect = TfidfVectorizer(analyzer='word', 
                             token_pattern=r'\w{1,}', 
                             max_features=2000,
                             min_df=0.01, 
                             max_df=0.95)

X_train_tfidf = tfidf_vect.fit_transform(X_train)

In [77]:
%%time
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', 
                                   token_pattern=r'\w{1,}', 
                                   ngram_range=(2,3), 
                                   max_features=2400,
                                   min_df=0.01, 
                                   max_df=0.95)
X_train_tfidf_ngram = tfidf_vect_ngram.fit_transform(X_train)

In [111]:
%%time
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', 
                                         token_pattern=r'\w{1,}', 
                                         ngram_range=(2,5), 
                                         max_features=10000,
                                         min_df=0.01, 
                                         max_df=0.95)
X_train_tfidf_ngram_chars = tfidf_vect_ngram_chars.fit_transform(X_train)

In [69]:
#Tranfroming the test data with TF-IDF Vectorizer
%%time
X_test_tfidf = tfidf_vect.transform(X_test)
X_test_tfidf_ngram = tfidf_vect_ngram.transform(X_test)
X_test_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(X_test)

In [34]:
#Converting y labels into series object for model fitting
y_test_en = pd.Series(y_test_en)
y_train_en = pd.Series(y_train_en)

### IMPLEMENTING COUNT VECTORIZER

In [142]:
#Creating count vectorizer object
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=1000, min_df=0.02, max_df=0.95)
count_vect.fit(df_all['review_text'])

# # transform the training and validation data using count vectorizer object
X_train_count =  count_vect.fit_transform(X_train)
X_test_count =  count_vect.transform(X_test)

## FITTING THE MODEL  AND FINDING ACCURACY

### TF-IDF VECTORIZER


In [124]:
#Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), X_train_tfidf, y_train_en, X_test_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

NB, WordLevel TF-IDF:  0.395


In [125]:
#Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), X_train_tfidf_ngram , y_train_en, X_test_tfidf_ngram )
print ("NB, N-Gram Vectors: ", accuracy)

NB, N-Gram Vectors:  0.3769


In [126]:
# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), X_train_tfidf_ngram_chars, y_train_en, X_test_tfidf_ngram_chars )
print ("NB, CharLevel Vectors: ", accuracy)

NB, CharLevel Vectors:  0.3895


### COUNT VECTORIZER

In [143]:
# Naive Bayes on Count Vectorizer
accuracy = train_model(naive_bayes.MultinomialNB(), X_train_count, y_train_en, X_test_count)
print("RF, Count Vectors: ",accuracy)

RF, Count Vectors:  0.389


# 2. Neural Network

### IMPLEMENTING TF-IDF VECTORIZER

### This was done in multiple iterations with different parameter values to achieve maximum accuracy on the results

In [144]:
# word level tf-idf
%%time
tfidf_vec`t = TfidfVectorizer(analyzer='word', 
                             token_pattern=r'\w{1,}', 
                             max_features=3000,
                             min_df=0.01, 
                             max_df=0.95)

X_train_tfidf = tfidf_vect.fit_transform(X_train)

In [ ]:
# ngram level tf-idf 
%%time
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', 
                                   token_pattern=r'\w{1,}', 
                                   ngram_range=(2,3), 
                                   max_features=1500,
                                   min_df=0.01, 
                                   max_df=0.95)
X_train_tfidf_ngram = tfidf_vect_ngram.fit_transform(X_train)

In [ ]:
# characters level tf-idf
%%time

tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', 
                                         token_pattern=r'\w{1,}', 
                                         ngram_range=(2,4), 
                                         max_features=5600,
                                         min_df=0.01, 
                                         max_df=0.95)
X_train_tfidf_ngram_chars = tfidf_vect_ngram_chars.fit_transform(X_train)

In [ ]:
#Tranfroming the test data with TF-IDF Vectorizer
%%time
X_test_tfidf = tfidf_vect.transform(X_test)
X_test_tfidf_ngram = tfidf_vect_ngram.transform(X_test)
X_test_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(X_test)

In [146]:
# Creating model architecture to fit Neural Network
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

## FITTING THE MODEL  AND FINDING ACCURACY

### TF-IDF VECTORIZER

In [155]:
# word level tf-idf
classifier = create_model_architecture(X_train_tfidf.shape[1])
accuracy = train_model(classifier, X_train_tfidf, y_train_en, X_test_tfidf, is_neural_net=True)
print ("NN, WordLevel TF-IDF: ", accuracy)

Epoch 1/1
40000/40000 [==============================] - 4s 108us/step - loss: -36.9859
NN, WordLevel TF-IDF:  0.2015


In [156]:
# ngram level tf-idf
classifier = create_model_architecture(X_train_tfidf_ngram.shape[1])
accuracy = train_model(classifier, X_train_tfidf_ngram , y_train_en, X_test_tfidf_ngram, is_neural_net=True)
print ("NN, N-Gram Vectors: ", accuracy)

Epoch 1/1
40000/40000 [==============================] - 5s 113us/step - loss: -36.3661
NN, N-Gram Vectors:  0.2015


In [157]:
#ngram_chars level tf-idf
classifier = create_model_architecture( X_train_tfidf_ngram_chars.shape[1])
accuracy = train_model(classifier, X_train_tfidf_ngram_chars, y_train_en, X_test_tfidf_ngram_chars,is_neural_net=True )
print ("NN, CharLevel Vectors: ", accuracy)

Epoch 1/1
40000/40000 [==============================] - 32s 794us/step - loss: -38.8145
NN, CharLevel Vectors:  0.2015


# 3. Random Forest Classifier

## Implementing TF-IDF vectorizer to find maximum accuracy in a function

In [162]:
i=500
while i < 5001:
    tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=i, min_df=0.01, max_df=0.95)
    # # transform the training and validation data using count vectorizer object
    X_train_tfidf = tfidf_vect.fit_transform(X_train)
    X_test_tfidf = tfidf_vect.transform(X_test)

    accuracy = train_model(ensemble.RandomForestClassifier(), X_train_tfidf, y_train_en, X_test_tfidf)
    print("RF, Count Vectors for max features=",i,":",accuracy)
    i=i+500

RF, Count Vectors for max features= 500 : 0.2955
RF, Count Vectors for max features= 1000 : 0.3334
RF, Count Vectors for max features= 1500 : 0.3191
RF, Count Vectors for max features= 2000 : 0.3124
RF, Count Vectors for max features= 2500 : 0.3217
RF, Count Vectors for max features= 3000 : 0.3182
RF, Count Vectors for max features= 3500 : 0.3219
RF, Count Vectors for max features= 4000 : 0.3218
RF, Count Vectors for max features= 4500 : 0.3152
RF, Count Vectors for max features= 5000 : 0.3221


### Implementing TF-IDF with 1000 features

In [ ]:
# Word level TF-IDF
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=1000, min_df=0.01, max_df=0.95
X_train_tfidf = tfidf_vect.fit_transform(X_train)

In [166]:
#Tranfroming the test data with TF-IDF Vectorizer 
X_test_tfidf = tfidf_vect.transform(X_test)

### IMPLEMENTING COUNT VECTORIZER

In [169]:
%%time

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=1000, min_df=0.01, max_df=0.95)

count_vect.fit(df_all['review_text'])

# # transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(X_train)
xvalid_count =  count_vect.transform(X_test)

CPU times: user 16.4 s, sys: 396 ms, total: 16.8 s
Wall time: 17.4 s


## FITTING THE MODEL  AND FINDING ACCURACY

### TF-IDF VECTORIZER

In [170]:
# word-level tf-idf
accuracy = train_model(ensemble.RandomForestClassifier(), X_train_tfidf, y_train_en, X_test_tfidf)
print("RF, WordLevel TF-IDF:",accuracy)

RF, WordLevel TF-IDF: 0.3212


### COUNT VECTORIZER

In [171]:
#  RF on Count Vectorizer
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, y_train_en, xvalid_count)
print("RF, Count Vectors: ",accuracy)

RF, Count Vectors:  0.3244


# 4. XG BOOST 

### Taking a CPU count for XGBoost Model to run at max processor efficiency

In [ ]:
import multiprocessing
n_jobs_cnt = multiprocessing.cpu_count()-1
n_jobs_cnt

### Defining XGBoost Classifier and using GridSearch with specific parameters

In [ ]:
xgb_clf = xgboost.XGBClassifier(n_estimators=300,
                                n_jobs=n_jobs_cnt)
param_grid_xgb = {'max_depth':[3,5],
                  'min_child_weight':[1,2],
                  'learning_rate':[0.05,0.1]}
grid_xgb = GridSearchCV(estimator=xgb_clf,
                        param_grid=param_grid_xgb,
                        verbose=10,
                        return_train_score=False,
                        scoring='accuracy',
                        cv=5)

### Fitting the XG Boost Model

In [ ]:
%%time
grid_xgb.fit(X=X_train_tfidf_ngram,
             y=y_train_en)

### Finding the best score, best parameter set and best estimator score out of the given set of outputs

In [ ]:
grid_xgb.best_score_, grid_xgb.best_params_

In [ ]:
grid_xgb.best_estimator_.score(X_test_tfidf_ngram,
                               y_test_en)